# Arize Tutorial: Boston House Prices

Let's get started on using Arize!✨

Arize helps you visualize your model performance, understand drift & data quality issues, and share insights learned from your models. 

In this tutorial, we will be building a model to predict Boston House Prices. The model is predicting a numeric value so we call it a ModelType.NUMERIC. After that, we will load the models's training inferences and test inferences into Arize. 🚀. 

### Running This Notebook
1. Click "Open in playground" to create a copy of this notebook for yourself.
2. Save a copy in Google Drive for yourself.
3. Step through each section below, pressing play on the code blocks to run the cells.
4. In Step 2, use your own Space and API key from your Arize account. 


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Arize-ai/client_python/blob/main/arize/examples/tutorials/Arize_Tutorial_Boston_House_Prices.ipynb)


## Step 1: Load Data and Build Model

In [ ]:
import pandas as pd
import uuid
import concurrent.futures as cf
from sklearn import ensemble
from sklearn import datasets
from sklearn.model_selection import train_test_split

###############################################################################
# 1 Load data and split data
data = datasets.load_boston()
X, y = datasets.load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, random_state=42
)

###############################################################################
# 2 Fit a regression model
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 0.5,
    "learning_rate": 0.01,
    "loss": "ls",
}
clf = ensemble.GradientBoostingRegressor(**params).fit(X_train, y_train)

# 3 Use the model to generate predictions
y_train_pred = clf.predict(X_train)
y_val_pred = clf.predict(X_val)
y_test_pred = clf.predict(X_test)

print("Step 1 ✅: Load Data & Build Model Done!")

## Step 2: Import and Setup Arize Client
You can find your `API KEY` and `SPACE ID` by navigating to the space settings page in your workspace (only space admins can see the keys). Copy those over to the set-up section. We will also be setting up some metadata to use across all logging.
<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-id-and-key.png" width="700">

In [ ]:
!pip install arize -q
from arize.api import Client
from arize.utils.types import ModelTypes

SPACE_ID = "SPACE_ID"
API_KEY = "API_KEY"
arize = Client(space_id=SPACE_ID, api_key=API_KEY)

# Saving model metadata for passing in later
model_id = "boston_house_prices"
model_version = "1.0"
model_type = ModelTypes.NUMERIC

print(
    "Step 2 ✅: Import and Setup Arize Client Done! Now we can start using Arize!"
)

## Step 3: Log Training Inferences to Arize
First step: Log the training data for your model to Arize! 

In [ ]:
# Logging training
train_prediction_labels = pd.Series(y_train_pred)
train_actual_labels = pd.Series(y_train)
train_feature_df = pd.DataFrame(X_train, columns=data["feature_names"])

train_responses = arize.log_training_records(
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,  # this will change depending on your model type
    prediction_labels=train_prediction_labels,
    actual_labels=train_actual_labels,
    features=train_feature_df,
)


## Helper to listen to response code to ensure successful delivery
def arize_responses_helper(responses):
    for response in cf.as_completed(responses):
        res = response.result()
        if res.status_code != 200:
            print(
                f"future failed with response code {res.status_code}, {res.text}"
            )


arize_responses_helper(train_responses)

print("Step 3 ✅: If no errors showed up, you have sent Training Inferences!")

## Step 4: Log Validation Inferences to Arize
Next Step: Log the validation data. You need to include `batch_id` to separate out different validation sets.

In [ ]:
# Logging validation
val_prediction_labels = pd.Series(y_val_pred)
val_actual_labels = pd.Series(y_val)
val_features_df = pd.DataFrame(X_val, columns=data["feature_names"])

val_responses = arize.log_validation_records(
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    batch_id="batch0",
    prediction_labels=val_prediction_labels,
    actual_labels=val_actual_labels,
    features=val_features_df,
)

arize_responses_helper(val_responses)
print("Step 4 ✅: If no errors showed up, you have sent Validation Inferences!")

## Step 5: Log Production Inferences to Arize  
Note: We will be sending our test data to emulate sending production data. 

1. **Time-Override:** You can directly specify the unix time which the prediction was made by using the optional argument `time_overwrite`. This will set the prediction timestamp. It can be a `pd.Series` or `List` of dtype `int` and the same length as the predictions sent. The timestamp at index 1 will be the prediction timestamp at index 1. You can set the prediction timestamp within the last year.

In our example, we used `time_overwrite` to simulate predictions over 30 days so you can see it displayed on arize platform right away.

2. For production, we use `log_bulk_predictions` 
and `log_bulk_actuals`. You can log the ground truth label after logging predictions at any time by matching the `predictions_ids` represented by `ids_df` here in this example.




In [ ]:
import datetime
import numpy as np

test_prediction_labels = pd.Series(y_test_pred)  # can also be pd.DataFrame()
test_actual_labels = pd.Series(y_test)
test_feature_df = pd.DataFrame(X_test, columns=data["feature_names"])

# ids_df will be used to match actuals after predictions have already been generated
ids_df = pd.DataFrame([str(uuid.uuid4()) for _ in range(len(y_test_pred))])

# OPTIONAL: Simulate predictions evenly distributed over 30 days by manually specifying prediction time
current_time = datetime.datetime.now().timestamp()
earlier_time = (
    datetime.datetime.now() - datetime.timedelta(days=30)
).timestamp()
optional_time_overwrite = np.linspace(
    earlier_time, current_time, num=len(y_test_pred)
)
optional_time_overwrite = pd.Series(optional_time_overwrite.astype(int))

# First we log the predictions. We are using log_bulk_predictions since we are sending more than 1 prediction.
prediction_responses = arize.log_bulk_predictions(
    model_id=model_id,
    model_version=model_version,
    model_type=model_type,
    prediction_ids=ids_df,
    prediction_labels=test_prediction_labels,
    features=test_feature_df,
    time_overwrite=optional_time_overwrite,  # this argument is optional
)

arize_responses_helper(prediction_responses)

print(
    "Step 5 ✅: If no errors showed up, you have successfully sent in {} Test Inferences!".format(
        len(optional_time_overwrite)
    )
)

## Step 6: Logging Production Actuals to Arize

We match the predictions above using `log_bulk_actuals` with the same `prediction_ids`. 

**Note:** You can use `log_bulk_predictions` for logging bulk predictions, and follow up with individual calls to the Arize API with `log_actual` instead of `log_bulk_actual` as labels become avaliable.

In [ ]:
# Next, we log the actuals. We are using log_bulk_actuals since we are sending more than 1 actual.
test_actual_labels = pd.Series(y_test)

actual_responses = arize.log_bulk_actuals(
    model_id=model_id,
    prediction_ids=ids_df,  # Pass in the same IDs to match the predictions & actuals.
    actual_labels=test_actual_labels,
)

arize_responses_helper(actual_responses)
print(
    "Step 6 ✅: If no errors showed up, you have successfully sent in Actuals!"
)

# Step 7: Generating and Formatting SHAP Values
**SHAP (SHapley Additive exPlanations)** is a game theoretic approach to explain the output of any machine learning model.

For more in-depth usage of the `shap` library, visit [SHAP Core Explainers](https://shap-lrjball.readthedocs.io/en/docs_update/generated/shap.Explainer.html) and pick an explainer specific to your machine learning model. `shap.Explainer` is the default explainer that will matches model type, but you can specify your own type. For example, you can choose to use `shap.TreeExplainer`, but it won't work on models such as `sklearn.LinearModel.LogisticRegression`.

We create this helper function `get_shap_values` to format the data and/or create visualizations for our shap values. We will store our results in a `pd.DataFrame` with matching columns for logging later.

In [ ]:
!pip install shap
import shap


def get_shap_values(
    model, X_data, ExplainerType=shap.Explainer, show_graph=False
):
    # NOTE: If there are errors, you  need to manually choose which explainer class
    explainer = ExplainerType(model, X_data)
    shap_values = explainer.shap_values(X_data)

    # When not in production, it can be helpful to check graphs for feature explainability
    if show_graph:
        shap.summary_plot(
            shap_values, X_data, feature_names=data["feature_names"]
        )

    # NOTE: Arize API expects a DataFrame of the same shape and column name as the model features.
    return pd.DataFrame(shap_values, columns=data["feature_names"])


shap_values = get_shap_values(clf, X, show_graph=True)
print(
    "Step 7 ✅: If no errors showed up, you should see a SHAP Graph above here!"
)

# Step 8: Logging SHAP Values to Arize
Similar to `log_bulk_actuals`, log the SHAP values using the `shap_df` our helper function generated in the previous sections.

In [ ]:
shap_df = get_shap_values(clf, X_test)  # custom function defined earlier

shap_responses = arize.log_bulk_shap_values(
    model_id=model_id, prediction_ids=ids_df, shap_values=shap_df
)

arize_responses_helper(shap_responses)
print("Step 8 ✅: If no errors showed up, you have sent SHAP values to Arize!")

## Workspace

Visit the [Arize Blog](https://arize.com/blog) and [Resource Center](https://arize.com/resource-hub/) for more resources on ML observability and model monitoring.

https://www.arize.com